## Import Statement

In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nnsplit import NNSplit
import pandas as pd 
import re

## Load data and get all the reviews

In [2]:
df = pd.read_csv("./data/RGeo_Hotel_Reviews.csv", index_col=0)

## Filter the columns and combine positive and negative review

In [3]:
df_negative_review = df[['Negative_Review', 'city', 'country']]
df_positive_review = df[['Positive_Review', 'city', 'country']]
df_uncleaned_reviews = pd.concat([df_negative_review, df_positive_review], ignore_index=True)

/Users/yuankanglee/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
df_uncleaned_reviews['Review'] = df_uncleaned_reviews['Negative_Review'].combine_first(df_uncleaned_reviews['Positive_Review'])
df_reviews = df_uncleaned_reviews.drop(columns=['Negative_Review', 'Positive_Review'])

## Shuffle and slice dataframe

In [6]:
df_reviews = df_reviews.sample(frac=1).reset_index(drop=True)
df_reviews[0:20000]

,city,country,Review
0,City of London,United Kingdom,The staff were amazing Always recognised us a...
1,Levallois-Perret,France,Bad wifi coverage in room
2,Lambeth,United Kingdom,Downstairs was always very busy nowhere to ha...
3,Kensington,United Kingdom,Excellent location next to earls court train ...
4,Vienna,Austria,Croissants are perfect and the breakfast in g...
...,...,...,...
19995,Clichy,France,Location and decor
19996,La Bordeta,Spain,only complaint really lack of electricity out...
19997,Barcelona,Spain,Breakfast didn t have much variety Bad smell ...
19998,City of London,United Kingdom,Excellent stay from start to finish Free drin...


## Splitting reviews in to sentence using NNSplit

In [ ]:
def split_review(reviews):
    reviews = [reviews.strip()]
    result_list = []
    splitter = NNSplit('en')
    results = splitter.split(reviews)
    for result in results[0]:
        sentence = ''
        for token in result:
            sentence += ' ' + token.text
        result_list.append(sentence.strip())
    return result_list      
        
df_reviews['sent_list'] = df_reviews['Review'].apply(split_review)

## Split the list into different rows of sentences in the dataframe

In [ ]:
s = df_reviews.apply(lambda x: pd.Series(x['sent_list']), axis=1).stack().reset_index(level=1, drop=True)

In [ ]:
s.name = 'sent_list'
df_reviews_sentence = df_reviews.drop('sent_list', axis=1).join(s)
df_reviews_sentence['sent_list'] = pd.Series(df_reviews_sentence['sent_list'], dtype=object)
df_reviews_sentence.reset_index(inplace=True)
df_reviews_sentence.drop(columns=['index', 'Review'])

## Filter sentence that is above length of 8

In [ ]:
length = (df_reviews_sentence['sent_list'].str.len() >= 8)
df_reviews_sentence = df_reviews_sentence.loc[length]

## Vader to tag the sentiment

In [ ]:
# This function gets the polarity of reviews using Vader # 
def get_polarity(sentence):
    analyser = SentimentIntensityAnalyzer()
    score = analyser.polarity_scores(sentence)
    compound = score['compound']
    if compound >= 0.05:
        return ('positive')
    elif compound > -0.05 and compound < 0.05: 
        return ('neutral')
    else: 
        return ('negative')
    
df_reviews_sentiment['polarity'] = df_reviews_sentence['review'].apply(get_polarity)